In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Pacakge Import

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.compose import ColumnTransformer,make_column_selector

from sklearn.metrics import classification_report,roc_auc_score,ConfusionMatrixDisplay,f1_score,accuracy_score
from sklearn.exceptions import NotFittedError

import tensorflow as tf

sns.set_style('darkgrid')

%matplotlib inline

# Data Import

In [ ]:
input_file='/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv'
df=pd.read_csv(input_file)
df.head()

# Null Heatmap

In [ ]:
plt.figure(figsize=(15,6))
sns.heatmap(df.isna(),yticklabels=False)

# EDA (Describe, Info)

In [ ]:
df.describe()

In [ ]:
df.info()

# EDA

In [ ]:
category_column=['sex', 'cp', 'fbs', 'restecg', 'exng', 'slp', 'caa', 'thall']

In [ ]:
for col in df.columns:
    print(f"{col:20} ----> {df[col].nunique():10} unique values")

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=df,x='output')
plt.tight_layout()

In [ ]:
fig_1,ax_1=plt.subplots(1,3,figsize=(18,4))
for i,col in enumerate(['sex','cp','fbs']):
    sns.countplot(data=df,x=col,ax=ax_1[i])
plt.tight_layout()

In [ ]:
fig_1,ax_1=plt.subplots(1,3,figsize=(18,4))
for i,col in enumerate(['restecg','exng','slp']):
    sns.countplot(data=df,x=col,ax=ax_1[i])
plt.tight_layout()

In [ ]:
fig_1,ax_1=plt.subplots(1,2,figsize=(12,4))
for i,col in enumerate(['caa','thall']):
    sns.countplot(data=df,x=col,ax=ax_1[i])
plt.tight_layout()

In [ ]:
plt.figure(figsize=(15,6))
sns.heatmap(data=df.corr(),cmap='cividis',annot=True)
plt.tight_layout()

# Data Preprocess Pipeline

In [ ]:
df.info()

In [ ]:
# for col in category_column:
#     df[col]=df[col].astype('object')

In [ ]:
# df.info()

In [ ]:
# plt.figure(figsize=(15,6))
# sns.heatmap(data=df.corr(),cmap='cividis',annot=True)
# plt.tight_layout()

In [ ]:
# cat_col_index=[1,2,5,6,8,10,11,12]
# num_col_index=[0,3,4,7,9]

In [ ]:
oh=OneHotEncoder(drop='first',handle_unknown='ignore')

ms,ss=MinMaxScaler(),StandardScaler()


ct=ColumnTransformer([
#     ('cat_encoder',oh,make_column_selector(dtype_include='object')),
    ('num_encoder',ss,[0,3,4,7,9])
],remainder='passthrough',verbose=True,n_jobs=-1)

# Separate X an y

In [ ]:
target='output'

In [ ]:
X=df.drop([target],axis=1)
y=df[[target]]
X.head(1)

In [ ]:
y.head(1)

# Train Test Split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,stratify=y,random_state=0)

In [ ]:
X_train.shape,y_train.shape

In [ ]:
X_test.shape,y_test.shape

In [ ]:
X_test.head(2)

In [ ]:
y_test.head(2)

# Data Preprocess

In [ ]:
X_train=ct.fit_transform(X_train)
X_test=ct.transform(X_test)

In [ ]:
print("-"*50)
for i in ct.transformers_:
    print(i)
    if(i[0]!='remainder'):
        print(i[1].get_feature_names_out())   
    print("-"*50)

In [ ]:
X_train.shape,y_train.shape

In [ ]:
tup1=None;tup2=None
try:
    tup1=(X_test[0].toarray(),y_test.head(1).values)  # scientific notation
    print("Sparse Matrix to Dense Array")
except:
    tup2=(X_test[0],y_test.head(1).values) 
    print("Normal Matrix to Dense Array")

tup1 if(tup1) else tup2

In [ ]:
X_test.shape,y_test.shape

# Model fitting

In [ ]:
def print_model_locations(model_list):
    print("-"*80)
    i=0
    for model in model_list:
        print(model)
        print(f"{model.__class__.__name__:30} at index {i} and suffix {i+1}")
        print("-"*80)
        i+=1

In [ ]:
def model_fitter(model_list):
    print("-"*50)
    for reg in model_list:
        reg_name=reg.__class__.__name__
        reg.fit(X_train,y_train.values.ravel())
        print(f"Fitting {reg_name} done")
        print("-"*50)

In [ ]:
from sklearn.linear_model import SGDClassifier,LogisticRegressionCV

from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier

from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier

from xgboost import XGBClassifier

from lightgbm import LGBMClassifier

In [ ]:
clf_1 = LogisticRegressionCV(n_jobs=-1,cv=5,max_iter=3000)
clf_8 =SGDClassifier(max_iter=3000)

clf_2 = SVC()

clf_3 = DecisionTreeClassifier()
clf_4 = ExtraTreeClassifier()

clf_5 = RandomForestClassifier(n_estimators=1000,n_jobs=-1)
clf_6 = GradientBoostingClassifier(n_estimators=1000,learning_rate=0.01) 
clf_7 = AdaBoostClassifier(base_estimator=RandomForestClassifier(n_jobs=-1),n_estimators=1000,learning_rate=0.01)


clf_9= XGBClassifier(n_estimators=1000,n_jobs=-1,learning_rate=0.01)
clf_10 = LGBMClassifier(n_estimators=1000,n_jobs=-1,learning_rate=0.01)

clf_list=[clf_3,clf_4,clf_5,clf_6,clf_7,clf_9,clf_10,clf_1,clf_2,clf_8]

In [ ]:
print_model_locations(clf_list)

In [ ]:
%%time
model_fitter(clf_list)

# Testing

In [ ]:
score_dict={}

In [ ]:
def tester_func(model_list):
    for i,clf in enumerate(model_list):
        
        clf_name = clf.__class__.__name__

        try:
            pred=clf.predict(X_test)
        except NotFittedError as e:
            print(repr(e))

        print("-"*80)
        print(f"Predicting {clf_name} done")

        score_dict[clf_name]={
            'roc_auc_score':roc_auc_score(y_test.values,pred),
            'f1_score':f1_score(y_test.values,pred),
            'accuracy_score':accuracy_score(y_test.values,pred),
            'model_index':int(i)
        }
    
    print("-"*80)

In [ ]:
tester_func(clf_list)

In [ ]:
score_pd=pd.DataFrame(score_dict).transpose().sort_values('roc_auc_score',ascending=False)
score_pd

In [ ]:
fig,ax=plt.subplots(1,3,figsize=(30,8))
for axi,col in enumerate(score_pd.columns[:-1]):
    sns.barplot(x=score_pd[col],y=score_pd.index,ax=ax[axi])
plt.tight_layout()

# Confusion Matrix for the top model

In [ ]:
score_pd.iloc[[0],:]

In [ ]:
win_index=int(score_pd.iloc[0,-1])
win_index

In [ ]:
sns.set_style('white')
ConfusionMatrixDisplay.from_predictions(y_test.values,clf_list[win_index].predict(X_test))